<a href="https://colab.research.google.com/github/aman-jn7/RAG-QA-Bot/blob/main/RAG_QA_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.2 MB/s eta 0:00:00


In [2]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.2 MB/s eta 0:00:00


In [3]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00


In [4]:
!pip install pyngrok

In [5]:
%%writefile app.py
import os
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pinecone import Pinecone, ServerlessSpec
import streamlit as st

# Initialize model and vector database
def initialize_components():
    # Load a pre-trained sentence transformer model
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Initialize Pinecone (vector database)
    pc = Pinecone(api_key="pcsk_3cYrjR_MA2LJ2KPMRRbYvg1AdqcZxFvNmwxqxVNzhMFekvw22NuAhVBGVn5kmwNpcfs7a5")

    # Specify a valid cloud and region for your Pinecone environment
    # Check Pinecone documentation or dashboard for supported clouds and regions
    index_name = "financial-data"
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=384,
            metric='cosine',
            spec=ServerlessSpec(cloud='aws', region='us-east-1')  # Ensure these are correct
        )
    index = pc.Index(index_name)

    return model, index

# Parse P&L data from PDFs
def parse_pdf_to_table(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()

    # Convert text to a structured table (basic example, extend as needed)
    data = []
    for line in text.split("\n"):
        if line.strip():
            data.append(line.split())

    df = pd.DataFrame(data)
    return df

def generate_embeddings(dataframe, model):
    terms = dataframe.apply(lambda row: " ".join(row.astype(str)), axis=1).tolist()
    embeddings = model.encode(terms, show_progress_bar=True)
    return terms, embeddings

def store_embeddings_in_pinecone(terms, embeddings, index):
    for i, (term, embedding) in enumerate(zip(terms, embeddings)):
        index.upsert([(f"id-{i}", embedding, {"text": term})])

def query_rag_model(query, model, index):
    query_embedding = model.encode(query)
    results = index.query(vector=query_embedding.tolist(), top_k=5, include_metadata=True)
    return results

# Build Streamlit interface
def build_streamlit_interface():
    st.title("Financial QA Bot")
    st.write("Upload a PDF containing P&L data and ask financial questions!")

    uploaded_file = st.file_uploader("Choose a PDF file", type="pdf")
    user_query = st.text_input("Enter your query")

    if uploaded_file and user_query:
        with st.spinner("Processing..."):
            model, index = initialize_components()
            df = parse_pdf_to_table(uploaded_file)
            terms, embeddings = generate_embeddings(df, model)
            store_embeddings_in_pinecone(terms, embeddings, index)
            results = query_rag_model(user_query, model, index)

            st.write("### Query Results")
            for match in results.matches:
                st.write(f"**Relevant Text:** {' '.join(filter(None, match.metadata['text'].split()))}")
                st.write(f"**Score:** {match.score:.4f}")

if __name__ == "__main__":
    build_streamlit_interface()

Writing app.py


In [6]:
from pyngrok import ngrok

In [7]:
!streamlit run app.py&>/dev/null&

In [8]:
!pgrep streamlit

1916


In [9]:
NGROK_AUTH_TOKEN = "2s23ZWDcUbzP9jEuvAl19LxmCTA_7RvpwG8nGJwuNvq4PRiD7"

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
publ_url = ngrok.connect(addr='8501')
print(publ_url)

NgrokTunnel: "https://e784-35-240-130-158.ngrok-free.app" -> "http://localhost:8501"
